In [1]:
import pandas as pd
import ast
import re

In [2]:
df = pd.read_csv("data/chords_and_lyrics.csv")

In [3]:
df

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,{3: 'G G/B ...,"{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,{1: 'Em D C C...,"{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
3,3,Justin Bieber,All Around The World,\nCapo on 4th fret\n\t \t\t\r\n\r\n \r\n\r\n ...,"{4: ' Intro: Em Bm Am C (2x) ', 6: ' ...","{0: '\nCapo on 4th fret\n\t \t\t', 1: '', 2: ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
4,4,Justin Bieber,All Around The World (acoustic),\n\t \t\t\r\n\r\nIntro: Gm - Dm - C - C x2 ...,"{2: 'Intro: Gm - Dm - C - C x2 ', 5: ' ...","{0: '\n\t \t\t', 1: '', 3: '', 4: 'verse 1: '...",{28: 'hide this tab e|--------------10--------...,en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135778,135778,Simon & Garfunkel,The Only Living Boy In New York,\n\t \t\tB E \r\nT...,{0: '\n\t \t\tB E ...,"{1: 'Tom, get your plane right on time ', 3: ""...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel
135779,135779,Simon & Garfunkel,The sound of silence,\nCapo on 6th fret\n\t \t\t \r\n\r\nIntro: A5...,"{2: 'Intro: A5(9) ', 5: 'A5(9) ...","{0: '\nCapo on 6th fret\n\t \t\t ', 1: '', 3:...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel
135780,135780,Simon & Garfunkel,The sounds of silence,\nCapo on 5th fret\n\t \t\t\r\n \r\nDm ...,"{2: 'Dm C ', 4: ' ...","{0: '\nCapo on 5th fret\n\t \t\t', 1: ' ', ...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel
135781,135781,Simon & Garfunkel,The Times They Are A - Changin,\n\t \t\tWritten by Bob Dylan. This is Simon ...,"{3: 'G Em ', 5: ' C G ',...","{0: ""\n\t \t\tWritten by Bob Dylan. This is S...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel


In [4]:
chord_regex = r'\b[A-G](?:#|b)?(?:m|maj|min|dim|aug|sus)?\*?\b'

In [5]:
# clean up chords
df["chords"] = df["chords"].str.findall(chord_regex)
df = df[df["chords"].str.len() > 0]


In [6]:
# create a column that turns it from list -> str
df = df.copy()
df["chords_str"] = df["chords"].apply(lambda x: " ".join(x))

In [7]:
df.iloc[3294].loc["chords_str"]

'C Am G F C Am G F C Am G F C Am G F C G Am F C G Am F Am F C G Am F C G F C Am G F C Am G F C G Am F C G Am F Am F C G Am F C G Am F C G Am F C G Am F C G Am F C G Am F C G Am F'

In [8]:
section_regex = re.compile(
    r'^(verse|chorus|pre-chorus|bridge|intro|outro|final|capo)\b',
    re.I
)

def clean_lyrics_cell(lyrics_str: str) -> str:
    # 1. string → dict
    lyrics_dict = ast.literal_eval(lyrics_str)

    cleaned_lines = []

    for line in lyrics_dict.values():
        if not line:
            continue

        line = line.strip()

        if not line:
            continue

        # remove section headers / capo notes
        if section_regex.match(line):
            continue

        # require at least 3 words
        if len(re.findall(r'\b\w+\b', line)) < 3:
            continue

        # remove parenthetical backing vocals
        line = re.sub(r'\([^)]*\)', '', line).strip()

        cleaned_lines.append(line)

    # 2. back to ONE string
    return "\n".join(cleaned_lines)


In [9]:
df["lyrics_clean"] = df["lyrics"].apply(clean_lyrics_cell)

In [10]:
df.iloc[0].loc["lyrics_clean"]

"Do you love the rain, does it make you dance\nWhen you're drunk with your friends at a party?\nWhat's your favorite song, does it make you smile?\nDo you think of me?\nWhen you close your eyes, tell me what are you dreamin'?\nEverything, I -- wanna know it all\nI'd spend ten thousand hours and ten thousand more\nOh if that's what it takes to learn that sweet heart of yours\nAnd I might never get there, but I'm gonna try\nIf it's ten thousand hours or the rest of my life\nI'm gonna love you\nDo you miss the road that you grew up on?\nDid you get your middle name from your grandma?\nWhen you think about your forever now\nDo you think of me?\nWhen you close your eyes, tell me what are you dreamin'?\nEverything, I -- wanna know it all\nI'd spend ten thousand hours and ten thousand more\nOh if that's what it takes to learn that sweet heart of yours\nAnd I might never get there, but I'm gonna try\nIf it's ten thousand hours or the rest of my life\nI'm gonna love you\nI'm gonna love you\nOoh

In [11]:
df

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords,chords_str,lyrics_clean
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,"[G, G, B, C, G, G, G, B, C, G, G, Em, C, G, G,...","{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,G G B C G G G B C G G Em C G G Em C G G Em C G...,"Do you love the rain, does it make you dance\n..."
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"[F#, F#, F#, E, F#, A, C, E, E, F#, Bm, A, C, ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,F# F# F# E F# A C E E F# Bm A C E F# A C E F# ...,Maybe i stare too much\nMaybe not long enough\...
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,"[Em, D, C, C, D, Em, Em, D, C, C, D, Em, Em, D...","{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,Em D C C D Em Em D C C D Em Em D C Am D Em G C...,No limit in the sky that I won't fly for ya\nN...
3,3,Justin Bieber,All Around The World,\nCapo on 4th fret\n\t \t\t\r\n\r\n \r\n\r\n ...,"[Em, Bm, Am, C, Em, Bm, Am, C, Em, Bm, Am, C, ...","{0: '\nCapo on 4th fret\n\t \t\t', 1: '', 2: ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,Em Bm Am C Em Bm Am C Em Bm Am C Bm Em Bm Am C...,"You're beautiful, beautiful you should know it..."
4,4,Justin Bieber,All Around The World (acoustic),\n\t \t\t\r\n\r\nIntro: Gm - Dm - C - C x2 ...,"[Gm, Dm, C, C, Gm, Dm, C, C, Gm, Dm, C, C, Gm,...","{0: '\n\t \t\t', 1: '', 3: '', 4: 'verse 1: '...",{28: 'hide this tab e|--------------10--------...,en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,Gm Dm C C Gm Dm C C Gm Dm C C Gm A C Gm A C Gm...,"You're beautiful, beautiful you should know it..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135778,135778,Simon & Garfunkel,The Only Living Boy In New York,\n\t \t\tB E \r\nT...,"[E, B, E, B, E, F, E, B, F, E, B, E, B, E, B, ...","{1: 'Tom, get your plane right on time ', 3: ""...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel,E B E B E F E B F E B E B E B E B E B F E C#m ...,"Tom, get your plane right on time\nI know, you..."
135779,135779,Simon & Garfunkel,The sound of silence,\nCapo on 6th fret\n\t \t\t \r\n\r\nIntro: A5...,"[G, Am, C, F, C, C, G, F, C, C, G, F, C, Am, C...","{0: '\nCapo on 6th fret\n\t \t\t ', 1: '', 3:...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel,G Am C F C C G F C C G F C Am C G Am G Am C F ...,"Hello, darkness, my old friend\nI've come to t..."
135780,135780,Simon & Garfunkel,The sounds of silence,\nCapo on 5th fret\n\t \t\t\r\n \r\nDm ...,"[Dm, C, Dm, F, Bb, F, Bb, F, Bb, F, Dm, F, C, ...","{0: '\nCapo on 5th fret\n\t \t\t', 1: ' ', ...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel,Dm C Dm F Bb F Bb F Bb F Dm F C Dm Dm C Dm F B...,"Hello darkness, my old friend,\nI've come to t..."
135781,135781,Simon & Garfunkel,The Times They Are A - Changin,\n\t \t\tWritten by Bob Dylan. This is Simon ...,"[G, Em, C, G, G, Am, C, D, G, Em, C, G, G, Am,...","{0: ""\n\t \t\tWritten by Bob Dylan. This is S...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel,G Em C G G Am C D G Em C G G Am D G D C D G C ...,Written by Bob Dylan. This is Simon & Garfunke...


In [12]:
# mapping chord progressions to roman numerals
NOTE_MAP = {
    "C": 0, "B#": 0,
    "C#": 1, "Db": 1,
    "D": 2,
    "D#": 3, "Eb": 3,
    "E": 4, "Fb": 4,
    "F": 5, "E#": 5,
    "F#": 6, "Gb": 6,
    "G": 7,
    "G#": 8, "Ab": 8,
    "A": 9,
    "A#": 10, "Bb": 10,
    "B": 11, "Cb": 11
}


NUM_TO_NOTE = {v: k for k, v in NOTE_MAP.items() if len(k) == 1 or "#" in k}

MAJOR_INTERVALS = [0, 2, 4, 5, 7, 9, 11]
MINOR_INTERVALS = [0, 2, 3, 5, 7, 8, 10]

MAJOR_ROMANS = ["I", "ii", "iii", "IV", "V", "vi", "vii°"]
MINOR_ROMANS = ["i", "ii°", "III", "iv", "v", "VI", "VII"]

CHORD_REGEX = re.compile(
    r"""(?x)
    (?P<root>[A-G](?:\#|b)?)        # root note
    (?P<quality>m|maj|dim|aug)?     # chord quality
    (?P<ext>(?:\d+|sus\d*|add\d+|maj7|m7|7)*)
    (?:/(?P<bass>[A-G](?:\#|b)?))?
    """
)


def parse_chord(chord: str):
    m = CHORD_REGEX.match(chord)
    if not m:
        return None, None

    root = m.group("root")
    quality = m.group("quality")

    if quality == "m":
        quality = "minor"
    elif quality == "dim":
        quality = "dim"
    elif quality == "aug":
        quality = "aug"
    else:
        quality = "major"

    return root, quality


def build_scale_pcs(key, mode):
    root = NOTE_MAP[key]
    intervals = MAJOR_INTERVALS if mode == "major" else MINOR_INTERVALS
    return {(root + i) % 12 for i in intervals}

def score_key(chords, key, mode):
    scale = build_scale_pcs(key, mode)
    score = 0

    for c in chords:
        root, _ = parse_chord(c)
        if root is None:
            continue

        pc = NOTE_MAP[root]
        if pc in scale:
            score += 1
        else:
            score -= 1

    # bonus if last chord is tonic
    last_root, _ = parse_chord(chords[-1])
    if last_root and NOTE_MAP[last_root] == NOTE_MAP[key]:
        score += 2

    return score

def infer_key(prog):
    chords = prog.split()
    best = None
    best_score = float("-inf")

    for key in NOTE_MAP:
        for mode in ["major", "minor"]:
            s = score_key(chords, key, mode)
            if s > best_score:
                best_score = s
                best = (key, mode)

    return best


In [13]:
df[["key", "mode"]] = pd.DataFrame(
    df["chords_str"].apply(infer_key).tolist(),
    index=df.index
)


In [14]:
df

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords,chords_str,lyrics_clean,key,mode
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,"[G, G, B, C, G, G, G, B, C, G, G, Em, C, G, G,...","{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,G G B C G G G B C G G Em C G G Em C G G Em C G...,"Do you love the rain, does it make you dance\n...",G,major
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"[F#, F#, F#, E, F#, A, C, E, E, F#, Bm, A, C, ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,F# F# F# E F# A C E E F# Bm A C E F# A C E F# ...,Maybe i stare too much\nMaybe not long enough\...,E,minor
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,"[Em, D, C, C, D, Em, Em, D, C, C, D, Em, Em, D...","{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,Em D C C D Em Em D C C D Em Em D C Am D Em G C...,No limit in the sky that I won't fly for ya\nN...,C,major
3,3,Justin Bieber,All Around The World,\nCapo on 4th fret\n\t \t\t\r\n\r\n \r\n\r\n ...,"[Em, Bm, Am, C, Em, Bm, Am, C, Em, Bm, Am, C, ...","{0: '\nCapo on 4th fret\n\t \t\t', 1: '', 2: ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,Em Bm Am C Em Bm Am C Em Bm Am C Bm Em Bm Am C...,"You're beautiful, beautiful you should know it...",E,minor
4,4,Justin Bieber,All Around The World (acoustic),\n\t \t\t\r\n\r\nIntro: Gm - Dm - C - C x2 ...,"[Gm, Dm, C, C, Gm, Dm, C, C, Gm, Dm, C, C, Gm,...","{0: '\n\t \t\t', 1: '', 3: '', 4: 'verse 1: '...",{28: 'hide this tab e|--------------10--------...,en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber,Gm Dm C C Gm Dm C C Gm Dm C C Gm A C Gm A C Gm...,"You're beautiful, beautiful you should know it...",G,major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135778,135778,Simon & Garfunkel,The Only Living Boy In New York,\n\t \t\tB E \r\nT...,"[E, B, E, B, E, F, E, B, F, E, B, E, B, E, B, ...","{1: 'Tom, get your plane right on time ', 3: ""...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel,E B E B E F E B F E B E B E B E B E B F E C#m ...,"Tom, get your plane right on time\nI know, you...",C,major
135779,135779,Simon & Garfunkel,The sound of silence,\nCapo on 6th fret\n\t \t\t \r\n\r\nIntro: A5...,"[G, Am, C, F, C, C, G, F, C, C, G, F, C, Am, C...","{0: '\nCapo on 6th fret\n\t \t\t ', 1: '', 3:...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel,G Am C F C C G F C C G F C Am C G Am G Am C F ...,"Hello, darkness, my old friend\nI've come to t...",G,minor
135780,135780,Simon & Garfunkel,The sounds of silence,\nCapo on 5th fret\n\t \t\t\r\n \r\nDm ...,"[Dm, C, Dm, F, Bb, F, Bb, F, Bb, F, Dm, F, C, ...","{0: '\nCapo on 5th fret\n\t \t\t', 1: ' ', ...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel,Dm C Dm F Bb F Bb F Bb F Dm F C Dm Dm C Dm F B...,"Hello darkness, my old friend,\nI've come to t...",D,minor
135781,135781,Simon & Garfunkel,The Times They Are A - Changin,\n\t \t\tWritten by Bob Dylan. This is Simon ...,"[G, Em, C, G, G, Am, C, D, G, Em, C, G, G, Am,...","{0: ""\n\t \t\tWritten by Bob Dylan. This is S...",{},en,70cRZdQywnSFp9pnc2WTCE,3354252.0,"['classic rock', 'folk', 'folk rock', 'melanch...",76,simon-&-garfunkel,G Em C G G Am C D G Em C G G Am D G D C D G C ...,Written by Bob Dylan. This is Simon & Garfunke...,G,major


In [15]:
df.to_csv("cleaned_dataset.csv", index=False)